In [1]:

import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.ensemble import ExtraTreesRegressor

In [2]:
%matplotlib inline
pd.options.display.max_columns = 45
pd.options.display.max_rows = 70

In [15]:

data = pd.read_csv("datapart2/historical_data1_Q12005.txt",sep="|",header=None)
data.columns = ["CREDIT SCORE","FIRST PAYMENT DATE","FIRST TIME HOMEBUYER FLAG","MATURITY DATE","METROPOLITAN STATISTICAL AREA",
               "MORTGAGE INSURANCE PERCENTAGE","NUMBER OF UNITS","OCCUPANCY STATUS","ORIGINAL COMBINED LOAN-TO-VALU","ORIGINAL DEBT-TO-INCOME (DTI) RATIO",
               "ORIGINAL UPB","ORIGINAL LOAN-TO-VALUE","ORIGINAL INTEREST RATE","CHANNE","PREPAYMENT PENALTY MORTGAGE FLAG",
               "PRODUCT TYPE","PROPERTY STATE","PROPERTY TYPE","POSTAL CODE","LOAN SEQUENCE NUMBER","LOAN PURPOSE",
               "ORIGINAL LOAN TERM","NUMBER OF BORROWERS","SELLER NAME","SERVICER NAME","SUPER CONFORMING FLAG"
                #,"Pre-HARP LOAN SEQUENCE NUMBER"
                ]
data.head()

,CREDIT SCORE,FIRST PAYMENT DATE,FIRST TIME HOMEBUYER FLAG,MATURITY DATE,METROPOLITAN STATISTICAL AREA,MORTGAGE INSURANCE PERCENTAGE,NUMBER OF UNITS,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL DEBT-TO-INCOME (DTI) RATIO,ORIGINAL UPB,ORIGINAL LOAN-TO-VALUE,ORIGINAL INTEREST RATE,CHANNE,PREPAYMENT PENALTY MORTGAGE FLAG,PRODUCT TYPE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN SEQUENCE NUMBER,LOAN PURPOSE,ORIGINAL LOAN TERM,NUMBER OF BORROWERS,SELLER NAME,SERVICER NAME,SUPER CONFORMING FLAG
0,699,200505,N,203504,39300.0,0,1,P,56,42,190000,56,5.625,R,N,FRM,RI,SF,2800.0,F105Q1000001,C,360,2,Other sellers,USBANKNA,NaN
1,691,200504,N,203503,36420.0,25,1,P,90,36,90000,90,5.750,R,N,FRM,OK,SF,73000.0,F105Q1000002,N,360,1,Other sellers,Other servicers,NaN
2,713,200503,N,203502,28740.0,0,1,P,72,45,357000,72,6.000,R,N,FRM,NY,SF,12500.0,F105Q1000003,P,360,2,Other sellers,Other servicers,NaN
3,719,200505,N,203504,NaN,0,1,S,85,47,195000,68,5.750,R,N,FRM,MO,CO,65000.0,F105Q1000004,P,360,2,Other sellers,USBANKNA,NaN
4,656,200503,N,203502,40340.0,0,1,P,68,30,253000,68,5.625,R,N,FRM,MN,SF,55900.0,F105Q1000005,C,360,2,Other sellers,Other servicers,NaN


In [16]:
def preprocessing(data):
    data['POSTAL CODE'].fillna(85200.0,inplace=True)
    data['PREPAYMENT PENALTY MORTGAGE FLAG'].fillna('N',inplace=True)
    data['METROPOLITAN STATISTICAL AREA'].fillna(16974.0,inplace=True)
    data.drop(['SUPER CONFORMING FLAG'],axis=1,inplace=True)
    data['CREDIT SCORE'].replace(to_replace=9999, value=np.nan, inplace=True)
    data['CREDIT SCORE'].fillna((data['CREDIT SCORE'].mean()), inplace=True)
    data['FIRST TIME HOMEBUYER FLAG'].replace(to_replace='9', value=np.nan, inplace=True)
    data['FIRST TIME HOMEBUYER FLAG'].fillna('N', inplace=True)
    data['Maturity_year'] =  [int(str(d)[:4]) for d in data['MATURITY DATE']]
    del data['MATURITY DATE']
    data['MORTGAGE INSURANCE PERCENTAGE'].replace(to_replace=999,value=0,inplace=True)
    data['NUMBER OF UNITS'].replace(to_replace=99,value=1,inplace=True)

    data['ORIGINAL COMBINED LOAN-TO-VALU'].replace(to_replace=999,value=80,inplace=True)

    data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].replace(to_replace=999,value=float('nan'),inplace=True)
    data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].fillna(data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].mean(),inplace=True)

    data['ORIGINAL LOAN-TO-VALUE'].replace(to_replace=999,value=float('nan'),inplace=True)
    data['ORIGINAL LOAN-TO-VALUE'].fillna(data['ORIGINAL LOAN-TO-VALUE'].mean(),inplace=True)

    data.drop('PRODUCT TYPE',axis=1,inplace=True)
    data.drop('LOAN SEQUENCE NUMBER',axis=1,inplace=True)

    data['PROPERTY TYPE'].replace(to_replace=99,value='SF',inplace=True)
    data['LOAN PURPOSE'].replace(to_replace=9,value='C',inplace=True)
    data['NUMBER OF BORROWERS'].replace(to_replace=99,value=2,inplace=True)

    del data['FIRST PAYMENT DATE']

    data.drop('ORIGINAL LOAN-TO-VALUE',axis=1,inplace=True)
    data.drop('Maturity_year',axis=1,inplace=True)

    #dictionary for each column
    cleanup_nums = {"FIRST TIME HOMEBUYER FLAG":{"Y": 1, "N": 0},
                "OCCUPANCY STATUS": {"P": 1, "S": 2, "I": 3 },
                "CHANNE":{"T": 1, "R":2,"C":3,"B":4},
                "PREPAYMENT PENALTY MORTGAGE FLAG":{"Y":1,"N":0},
                "PROPERTY TYPE":{"SF":1,"PU":2,"CO":3,"MH":4,"CP":5,"99":99},
                "LOAN PURPOSE":{"C":1,"P":2,"N":3},
                "SELLER NAME":{"Other sellers": 0, "COUNTRYWIDE": 1, "TAYLOR,BEAN&WHITAKER": 2, "PROVIDENTFUNDINGASSO": 3, "USBANKNA": 4, "FIFTHTHIRDBANK": 5, "ABNAMROMTGEGROUP,INC": 6, "CHASEHOMEFINANCELLC": 7, "NATLCITYMTGECO": 8, "WELLSFARGOBANK,NA": 9, "GMACMTGECORP": 10, "WASHINGTONMUTUALBANK": 11, "FLAGSTARBANK,FSB": 12, "BANKOFAMERICA,NA": 13},
                "PREPAYMENT PENALTY MORTGAGE FLAG":{"Y": 1, "N": 0},
                "PROPERTY STATE":{"RI": 0, "OK": 1, "NY": 2, "MO": 3, "MN": 4, "IL": 5, "KY": 6, "WA": 7, "TX": 8, "FL": 9, "CA": 10, "IN": 11, "NJ": 12, "ID": 13, "TN": 14, "KS": 15, "MI": 16, "IA": 17, "MT": 18, "GA": 19, "OH": 20, "OR": 21, "ME": 22, "CT": 23, "WV": 24, "NH": 25, "VA": 26, "NC": 27, "AZ": 28, "NE": 29, "MD": 30, "MA": 31, "UT": 32, "CO": 33, "ND": 34, "PA": 35, "SC": 36, "DE": 37, "SD": 38, "WI": 39, "AL": 40, "AK": 41, "VT": 42, "LA": 43, "AR": 44, "NM": 45, "HI": 46, "DC": 47, "MS": 48, "NV": 49, "GU": 50, "WY": 51, "PR": 52, "VI": 53},
                "SERVICER NAME":{"USBANKNA": 0, "Other servicers": 1, "PNCMTGESERVICES,INC": 2, "WELLSFARGOBANK,NA": 3, "WASHINGTONMUTUALBANK": 4, "PNCBANK,NATL": 5, "NATLCITYMTGECO": 6, "JPMORGANCHASEBANK,NA": 7, "NATIONSTARMTGELLCDBA": 8, "COUNTRYWIDE": 9, "BANKOFAMERICA,NA": 10, "BACHOMELOANSERVICING": 11, "CITIMORTGAGE,INC": 12, "PROVIDENTFUNDINGASSO": 13, "ABNAMROMTGEGROUP,INC": 14, "FIFTHTHIRDBANK": 15, "GMACMORTGAGE,LLC": 16},
                "OCCUPANCY STATUS":{"P": 1, "S": 2, "I": 3 },
                "CHANNE":{"T": 1, "R":2,"C":3,"B":4}  
               }
    data.replace(cleanup_nums, inplace=True)
    
    #d_seller = {"Other sellers": 0, "COUNTRYWIDE": 1, "TAYLOR,BEAN&WHITAKER": 2, "PROVIDENTFUNDINGASSO": 3, "USBANKNA": 4, "FIFTHTHIRDBANK": 5, "ABNAMROMTGEGROUP,INC": 6, "CHASEHOMEFINANCELLC": 7, "NATLCITYMTGECO": 8, "WELLSFARGOBANK,NA": 9, "GMACMTGECORP": 10, "WASHINGTONMUTUALBANK": 11, "FLAGSTARBANK,FSB": 12, "BANKOFAMERICA,NA": 13}
    #d_service  = {"USBANKNA": 0, "Other servicers": 1, "PNCMTGESERVICES,INC": 2, "WELLSFARGOBANK,NA": 3, "WASHINGTONMUTUALBANK": 4, "PNCBANK,NATL": 5, "NATLCITYMTGECO": 6, "JPMORGANCHASEBANK,NA": 7, "NATIONSTARMTGELLCDBA": 8, "COUNTRYWIDE": 9, "BANKOFAMERICA,NA": 10, "BACHOMELOANSERVICING": 11, "CITIMORTGAGE,INC": 12, "PROVIDENTFUNDINGASSO": 13, "ABNAMROMTGEGROUP,INC": 14, "FIFTHTHIRDBANK": 15, "GMACMORTGAGE,LLC": 16}
    #d_seller.setdefault(data["SELLER NAME"], default=0)
    #d_service.setdefault(data["SERVICER NAME"], default=0)
    print('>')
    data=data[['ORIGINAL INTEREST RATE','CREDIT SCORE','MORTGAGE INSURANCE PERCENTAGE','OCCUPANCY STATUS','ORIGINAL COMBINED LOAN-TO-VALU'
                ,'ORIGINAL UPB','CHANNE','PROPERTY STATE','PROPERTY TYPE','POSTAL CODE'
                ,'LOAN PURPOSE','ORIGINAL LOAN TERM','SELLER NAME','SERVICER NAME']]
    print('>')
    for i in range(len(data['SELLER NAME'])):
        if (not(str(data['SELLER NAME'][i]).isdigit())):
            data['SELLER NAME'][i]=0
    print('>')
    for i in range(len(data['SERVICER NAME'])):
        if (not(str(data['SERVICER NAME'][i]).isdigit())):
            data['SERVICER NAME'][i]=1 
    print('>')
    return data
    

In [17]:
data=preprocessing(data)

In [18]:
data.head()

,ORIGINAL INTEREST RATE,CREDIT SCORE,MORTGAGE INSURANCE PERCENTAGE,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL UPB,CHANNE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN PURPOSE,ORIGINAL LOAN TERM,SELLER NAME,SERVICER NAME
0,5.625,699.0,0,1,56,190000,2,0,1,2800.0,1,360,0,0
1,5.750,691.0,25,1,90,90000,2,1,1,73000.0,3,360,0,1
2,6.000,713.0,0,1,72,357000,2,2,1,12500.0,2,360,0,1
3,5.750,719.0,0,2,85,195000,2,3,3,65000.0,2,360,0,0
4,5.625,656.0,0,1,68,253000,2,4,1,55900.0,1,360,0,1


In [19]:
val = pd.read_csv("datapart2/historical_data1_Q22005.txt",sep="|",header=None)
val.columns = ["CREDIT SCORE","FIRST PAYMENT DATE","FIRST TIME HOMEBUYER FLAG","MATURITY DATE","METROPOLITAN STATISTICAL AREA",
               "MORTGAGE INSURANCE PERCENTAGE","NUMBER OF UNITS","OCCUPANCY STATUS","ORIGINAL COMBINED LOAN-TO-VALU","ORIGINAL DEBT-TO-INCOME (DTI) RATIO",
               "ORIGINAL UPB","ORIGINAL LOAN-TO-VALUE","ORIGINAL INTEREST RATE","CHANNE","PREPAYMENT PENALTY MORTGAGE FLAG",
               "PRODUCT TYPE","PROPERTY STATE","PROPERTY TYPE","POSTAL CODE","LOAN SEQUENCE NUMBER","LOAN PURPOSE",
               "ORIGINAL LOAN TERM","NUMBER OF BORROWERS","SELLER NAME","SERVICER NAME","SUPER CONFORMING FLAG"
                #,"Pre-HARP LOAN SEQUENCE NUMBER"
                ]
val.head()

,CREDIT SCORE,FIRST PAYMENT DATE,FIRST TIME HOMEBUYER FLAG,MATURITY DATE,METROPOLITAN STATISTICAL AREA,MORTGAGE INSURANCE PERCENTAGE,NUMBER OF UNITS,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL DEBT-TO-INCOME (DTI) RATIO,ORIGINAL UPB,ORIGINAL LOAN-TO-VALUE,ORIGINAL INTEREST RATE,CHANNE,PREPAYMENT PENALTY MORTGAGE FLAG,PRODUCT TYPE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN SEQUENCE NUMBER,LOAN PURPOSE,ORIGINAL LOAN TERM,NUMBER OF BORROWERS,SELLER NAME,SERVICER NAME,SUPER CONFORMING FLAG
0,715,200508,N,203507,33700.0,0,1,P,58,41,214000,58,5.750,R,N,FRM,CA,SF,95300.0,F105Q2000001,C,360,2,Other sellers,"WELLSFARGOBANK,NA",NaN
1,743,200508,N,203507,NaN,0,1,P,80,36,340000,80,5.875,R,N,FRM,IL,SF,62400.0,F105Q2000002,N,360,2,Other sellers,"PNCMTGESERVICES,INC",NaN
2,772,200508,N,203507,37860.0,0,1,P,80,34,184000,80,5.500,R,N,FRM,FL,PU,32500.0,F105Q2000003,P,360,1,Other sellers,Other servicers,NaN
3,773,200507,N,203506,NaN,0,1,P,64,60,260000,64,5.875,R,N,FRM,MN,SF,56300.0,F105Q2000004,N,360,2,Other sellers,USBANKNA,NaN
4,647,200508,N,203507,NaN,30,1,P,91,28,170000,91,6.000,R,N,FRM,MI,SF,49200.0,F105Q2000005,N,360,1,Other sellers,Other servicers,NaN


In [20]:
val=preprocessing(val)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
val.head()

,ORIGINAL INTEREST RATE,CREDIT SCORE,MORTGAGE INSURANCE PERCENTAGE,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL UPB,CHANNE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN PURPOSE,ORIGINAL LOAN TERM,SELLER NAME,SERVICER NAME
0,5.750,715.0,0,1,58,214000,2,10,1,95300.0,1,360,0,3
1,5.875,743.0,0,1,80,340000,2,5,1,62400.0,3,360,0,2
2,5.500,772.0,0,1,80,184000,2,9,2,32500.0,2,360,0,1
3,5.875,773.0,0,1,64,260000,2,4,1,56300.0,3,360,0,0
4,6.000,647.0,30,1,91,170000,2,16,1,49200.0,3,360,0,1


In [21]:
(train_x,train_y)=(data.drop('ORIGINAL INTEREST RATE',axis=1),data['ORIGINAL INTEREST RATE'])

In [22]:
(test_x,test_y)=(val.drop('ORIGINAL INTEREST RATE',axis=1),val['ORIGINAL INTEREST RATE'])

In [23]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [24]:
#gradient boosting
gbm_reg = GradientBoostingRegressor(alpha=0.85, learning_rate=0.1, loss="huber", max_depth=10, max_features=0.7, min_samples_leaf=18, min_samples_split=19, n_estimators=100, subsample=0.4,verbose=2)
gbm_reg.fit(train_x,train_y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0557           0.0047            3.96m
         2           0.0516           0.0045            3.89m
         3           0.0474           0.0037            3.85m
         4           0.0445           0.0031            3.90m
         5           0.0421           0.0025            3.86m
         6           0.0396           0.0021            3.83m
         7           0.0381           0.0018            3.83m
         8           0.0364           0.0015            3.78m
         9           0.0353           0.0011            3.76m
        10           0.0342           0.0009            3.77m
        11           0.0333           0.0008            3.73m
        12           0.0326           0.0007            3.68m
        13           0.0320           0.0006            3.62m
        14           0.0314           0.0005            3.56m
        15           0.0312           0.0005            3.53m
       

GradientBoostingRegressor(alpha=0.85, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=10,
             max_features=0.7, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=18, min_samples_split=19,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=0.4, tol=0.0001, validation_fraction=0.1, verbose=2,
             warm_start=False)

In [25]:
pred_y = gbm_reg.predict(test_x)
print("Score for train: "+str(gbm_reg.score(train_x,train_y)))
print("Score: "+str(gbm_reg.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

Score for train: 0.5623285677659279
Score: 0.2565487912049924
RMS: 0.3022910585787838
MAPE: 3.960806672468125
R2: 0.2565487912049924
MAE: 0.23215108590036815
